# 5th Assignment by UI17CO49

1. Write a CFG rule for a sentence.
2. Implement RegexParser and RecursiveDescentParser.
3. Implement chart parser.
4. Implement probabilistic CFG using an inside chart parser.
5. Implement probabilistic CFG using the viterbi algorithm.
6. Draw a parse tree.
7. Identify if a sentence is ambiguous or not.

## Importing library

In [157]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1. Creating a CFG

In [158]:
from nltk import Nonterminal, nonterminals, Production, CFG

In [159]:
nt1 = Nonterminal('NP')
nt2 = Nonterminal('VP')

In [160]:
nt1.symbol()

'NP'

In [161]:
S, NP, VP, PP = nonterminals('S, NP, VP, PP')
N, V, P, DT = nonterminals('N, V, P, DT')

In [162]:
prod1 = Production(S, [NP, VP])
prod2 = Production(NP, [DT, NP])

In [163]:
grammar = CFG.fromstring("""
    S -> NP VP
    PP -> P NP
    NP -> 'the' N | N PP | 'the' N PP
    VP -> V NP | V PP | V NP PP
    N -> 'cat' | 'dog' | 'rug'
    V -> 'chased' | 'sat'
    P -> 'in' | 'on'
""")

## 2. a. Implementing RecursiveDescentParser

In [164]:
from nltk.parse import RecursiveDescentParser

rdp = RecursiveDescentParser(grammar)

In [165]:
sent1 = "the cat chased the dog".split()
sent2 = "the cat chased the dog on the rug".split()

In [166]:
for t in rdp.parse(sent1):
    print(t)

(S (NP the (N cat)) (VP (V chased) (NP the (N dog))))


In [167]:
temp = 0
for t in rdp.parse(sent2):
    temp += 1
    print(t)

(S
  (NP the (N cat))
  (VP (V chased) (NP the (N dog) (PP (P on) (NP the (N rug))))))
(S
  (NP the (N cat))
  (VP (V chased) (NP the (N dog)) (PP (P on) (NP the (N rug)))))


## 2. b. Implementing RegexpParser

In [168]:
import matplotlib
matplotlib.use('Agg')

sentence = [
    ("the", "NP"),
    ("cat", "N"),
    ("chased", "V"),
    ("the", "NP"),
    ("dog", "N"),
    ("on", "P"),
    ("rug", "N")
]

grammar = "NP:{<DT>?<JJ>*<NN>}"

rp = nltk.RegexpParser(grammar)
op = rp.parse(sentence)
op

# an error will be generated because this notebook was made using 
# Google Colab and there is an issue regarding tkinter display variable on
# this platform.

TclError: ignored

Tree('S', [('the', 'NP'), ('cat', 'N'), ('chased', 'V'), ('the', 'NP'), ('dog', 'N'), ('on', 'P'), ('rug', 'N')])

## 3. Implementing a chart parser.

In [170]:
nltk.parse.chart.demo(2, print_times=False, trace=1, sent='I saw a dog', numparses=1)

* Sentence:
I saw a dog
['I', 'saw', 'a', 'dog']

* Strategy: Bottom-up

|.    I    .   saw   .    a    .   dog   .|
|[---------]         .         .         .| [0:1] 'I'
|.         [---------]         .         .| [1:2] 'saw'
|.         .         [---------]         .| [2:3] 'a'
|.         .         .         [---------]| [3:4] 'dog'
|>         .         .         .         .| [0:0] NP -> * 'I'
|[---------]         .         .         .| [0:1] NP -> 'I' *
|>         .         .         .         .| [0:0] S  -> * NP VP
|>         .         .         .         .| [0:0] NP -> * NP PP
|[--------->         .         .         .| [0:1] S  -> NP * VP
|[--------->         .         .         .| [0:1] NP -> NP * PP
|.         >         .         .         .| [1:1] Verb -> * 'saw'
|.         [---------]         .         .| [1:2] Verb -> 'saw' *
|.         >         .         .         .| [1:1] VP -> * Verb NP
|.         >         .         .         .| [1:1] VP -> * Verb
|.         [--------->

In [171]:
# top down
nltk.parse.chart.demo(1, print_times=False, trace=0, numparses=2,
                      sent='I saw John with a dog')

* Sentence:
I saw John with a dog
['I', 'saw', 'John', 'with', 'a', 'dog']

* Strategy: Top-down

Nr edges in chart: 48
(S
  (NP I)
  (VP (Verb saw) (NP (NP John) (PP with (NP (Det a) (Noun dog))))))
(S
  (NP I)
  (VP (VP (Verb saw) (NP John)) (PP with (NP (Det a) (Noun dog)))))



In [172]:
# bottom up
nltk.parse.chart.demo(2, print_times=False, trace=0, numparses=2,
                      sent='I saw John with a dog')

* Sentence:
I saw John with a dog
['I', 'saw', 'John', 'with', 'a', 'dog']

* Strategy: Bottom-up

Nr edges in chart: 53
(S
  (NP I)
  (VP (VP (Verb saw) (NP John)) (PP with (NP (Det a) (Noun dog)))))
(S
  (NP I)
  (VP (Verb saw) (NP (NP John) (PP with (NP (Det a) (Noun dog))))))



In [173]:
# stepping chart parser
nltk.parse.chart.demo(5, print_times=False, trace=1, numparses=2,
                      sent='I saw John with a dog')

* Sentence:
I saw John with a dog
['I', 'saw', 'John', 'with', 'a', 'dog']

* Strategy: Stepping (top-down vs bottom-up)

*** SWITCH TO TOP DOWN
|[------]      .      .      .      .      .| [0:1] 'I'
|.      [------]      .      .      .      .| [1:2] 'saw'
|.      .      [------]      .      .      .| [2:3] 'John'
|.      .      .      [------]      .      .| [3:4] 'with'
|.      .      .      .      [------]      .| [4:5] 'a'
|.      .      .      .      .      [------]| [5:6] 'dog'
|>      .      .      .      .      .      .| [0:0] S  -> * NP VP
|>      .      .      .      .      .      .| [0:0] NP -> * NP PP
|>      .      .      .      .      .      .| [0:0] NP -> * Det Noun
|>      .      .      .      .      .      .| [0:0] NP -> * 'I'
|[------]      .      .      .      .      .| [0:1] NP -> 'I' *
|[------>      .      .      .      .      .| [0:1] S  -> NP * VP
|[------>      .      .      .      .      .| [0:1] NP -> NP * PP
|.      >      .      .      .      .      .| [1

## 4. Implementing probabilistic CFG using InsideChartParse class.

In [174]:
nltk.download('treebank')
from nltk.corpus import treebank
from itertools import islice
from nltk.grammar import PCFG, induce_pcfg, toy_pcfg1, toy_pcfg2

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


In [175]:
grammar = PCFG.fromstring("""
    A -> B B [.3] | C B C [.7]
    B -> B D [.5] | C [.5]
    C -> 'a' [.1] | 'b' [0.9]
    D -> 'b' [1.0]
""")

prod = grammar.productions()[0]
prod

A -> B B [0.3]

In [176]:
prod.lhs(); prod.rhs(); prod.prob(); grammar.start(); grammar.productions()

[A -> B B [0.3],
 A -> C B C [0.7],
 B -> B D [0.5],
 B -> C [0.5],
 C -> 'a' [0.1],
 C -> 'b' [0.9],
 D -> 'b' [1.0]]

In [177]:
productions = []
for fileid in treebank.fileids()[:2]:
    for t in treebank.parsed_sents(fileid):
        productions += t.productions()

In [178]:
grammar = induce_pcfg(S, productions)
grammar

<Grammar with 71 productions>

In [179]:
sorted(grammar.productions(lhs=Nonterminal('PP')))[:2]

[PP -> IN NP [1.0]]

In [180]:
sorted(grammar.productions(lhs=Nonterminal('NNP')))[:2]

[NNP -> 'Agnew' [0.0714286], NNP -> 'Consolidated' [0.0714286]]

In [181]:
sorted(grammar.productions(lhs=Nonterminal('JJ')))[:2]

[JJ -> 'British' [0.142857], JJ -> 'former' [0.142857]]

In [182]:
sorted(grammar.productions(lhs=Nonterminal('NP')))[:2]

[NP -> CD NNS [0.133333], NP -> DT JJ JJ NN [0.0666667]]

In [183]:
## custom sentence

tagged_sentence = "Jack saw Bob with my cookie".split()
grammar = toy_pcfg2
print(grammar)

Grammar with 23 productions (start state = S)
    S -> NP VP [1.0]
    VP -> V NP [0.59]
    VP -> V [0.4]
    VP -> VP PP [0.01]
    NP -> Det N [0.41]
    NP -> Name [0.28]
    NP -> NP PP [0.31]
    PP -> P NP [1.0]
    V -> 'saw' [0.21]
    V -> 'ate' [0.51]
    V -> 'ran' [0.28]
    N -> 'boy' [0.11]
    N -> 'cookie' [0.12]
    N -> 'table' [0.13]
    N -> 'telescope' [0.14]
    N -> 'hill' [0.5]
    Name -> 'Jack' [0.52]
    Name -> 'Bob' [0.48]
    P -> 'with' [0.61]
    P -> 'under' [0.39]
    Det -> 'the' [0.41]
    Det -> 'a' [0.31]
    Det -> 'my' [0.28]


In [184]:
from nltk.parse import pchart

parser = pchart.InsideChartParser(grammar)
for t in parser.parse(tagged_sentence):
    print(t)

(S
  (NP (Name Jack))
  (VP
    (V saw)
    (NP
      (NP (Name Bob))
      (PP (P with) (NP (Det my) (N cookie)))))) (p=6.31607e-06)
(S
  (NP (Name Jack))
  (VP
    (VP (V saw) (NP (Name Bob)))
    (PP (P with) (NP (Det my) (N cookie))))) (p=2.03744e-07)


## 5. Implementing probabilistic CFG using Viterbi Parse Class

In [185]:
from nltk.parse import ViterbiParser
tagged_sentence = "Jack saw Bob with my cookie under the hill".split()
grammar = toy_pcfg2

In [186]:
parser = ViterbiParser(grammar)
for t in parser.parse(tagged_sentence):
    print(t)

(S
  (NP (Name Jack))
  (VP
    (V saw)
    (NP
      (NP
        (NP (Name Bob))
        (PP (P with) (NP (Det my) (N cookie))))
      (PP (P under) (NP (Det the) (N hill)))))) (p=6.41816e-08)


## 6. Draw a tree

In [187]:
print(tagged_sentence, end="\n\n")
for t in list(parser.parse(tagged_sentence)):
    print(t)

['Jack', 'saw', 'Bob', 'with', 'my', 'cookie', 'under', 'the', 'hill']

(S
  (NP (Name Jack))
  (VP
    (V saw)
    (NP
      (NP
        (NP (Name Bob))
        (PP (P with) (NP (Det my) (N cookie))))
      (PP (P under) (NP (Det the) (N hill)))))) (p=6.41816e-08)


#### To draw a tree, we need to store the values returned by parser.parse(tagged_sentence) in a variable _x_ and call the function _draw()_ on _x_ by writing _x.draw()_. Google colab doesn't have support for tkinter variables hence tree can't be made and a localhost jupyter connection is needed. 

## 7. Identify if sentence is ambiguous or not.

In [189]:
grammar = CFG.fromstring("""
    S -> NP VP
    PP -> P NP
    NP -> 'the' N | N PP | 'the' N PP
    VP -> V NP | V PP | V NP PP
    N -> 'cat' | 'dog' | 'rug'
    V -> 'chased' | 'sat'
    P -> 'in' | 'on'
""")

In [190]:
sent1 = "the cat chased the dog".split()
sent2 = "the cat chased the dog on the rug".split()

parser = RecursiveDescentParser(grammar)

In [193]:
count = 0

for _ in parser.parse(sent1):
    count += 1

if count > 1 :
    print("The sentence", " ".join(sent1), "is AMBIGUOUS!")
else:
    print("The sentence", " ".join(sent1), "is NOT AMBIGUOUS!")

The sentence the cat chased the dog is NOT AMBIGUOUS!


In [194]:
count = 0

for _ in parser.parse(sent2):
    count += 1

if count > 1 :
    print("The sentence", " ".join(sent2), "is AMBIGUOUS!")
else:
    print("The sentence", " ".join(sent2), "is NOT AMBIGUOUS!")

The sentence the cat chased the dog on the rug is AMBIGUOUS!
